# Investigating large forces in HMC

In [ ]:
from math import pi as π

import torch
import pandas as pd

from nflows_xy.core import FlowBasedSampler, PullbackAction
from nflows_xy.flows import AutoregressiveFlow
from nflows_xy.plot import plot_training_metrics
from nflows_xy.train import train, test
from nflows_xy.xy import action
from nflows_xy.utils import mod_2pi

## Data generation

### Meshgrid

### Scatter

### Molecular Dynamics trajectory

In [ ]:
@torch.no_grad()
def leapfrog(
    z0,
    p0,
    action,
    step_size,
    traj_length,
):
    """Similar to nflows_xy.hmc.leapfrog but tracks quantities at each step."""

    n_steps = max(1, round(traj_length / abs(step_size)))

    t = 0.
    z = z0.clone()
    p = p0.clone()
    ε = step_size
    F = action.grad(z).negative()

    z_ = [z.clone()]
    p_ = [p.clone()]
    t_ = [t]

    for _ in range(n_steps):     
       
        p = p + (ε / 2) * F
        z = mod_2pi(z + ε * p)
        F = action.grad(z).negative()
        p = p + (ε / 2) * F
        t += ε
        
        z_.append(z.clone())
        p_.append(p.clone())
        t_.append(t)

    t = torch.tensor(t_)
    z = torch.cat(z_)
    p = torch.cat(p_)
    S_pull = action(z)
    F = action.grad(z).negative()
    φ, ldj = action.flow(z)
    S_targ = action.target(φ)
    H = (p**2 / 2).sum(dim=1) + S_pull 
    
    data = torch.cat(
        [
            z.squeeze(-1),
            φ.squeeze(-1),
            p.squeeze(-1),
            F.squeeze(-1),
            F.pow(2).sum(1).sqrt(),
            ldj,
            S_pull,
            S_targ,
            H,
        ],
        dim=1,
    )
    columns = [
        *[f"z{i}" for i in range(L)],
        *[f"φ{i}" for i in range(L)],
        *[f"p{i}" for i in range(L)],
        *[f"F{i}" for i in range(L)],
        "|F|",
        "ldj",
        "S_pull",
        "S_targ",
        "H",
    ]
    
    trajectory = pd.DataFrame(
        data=data,
        index=time.numpy(),
        columns=columns,
    )

    return trajectory

## Demonstration of the problem

In [ ]:
L = 2

target = action(beta=3.0, lattice_size=L, lattice_dim=1)
flow = AutoregressiveFlow(
    lattice_size=L,
    n_mixture=12,
    net_shape=[32],
    net_activation="ReLU",
)
model = FlowBasedSampler(flow, target)

training_metrics = train(
    model, 
    n_steps=2000,
    batch_size=2048,
)

training_metrics.plot(x="step", y=["loss", "ess", "vlw"], subplots=True)

In [ ]:
pullback = PullbackAction(flow=model.flow, target=model.target)
trajectory = leapfrog(
    z0=torch.empty(1, L, 1).uniform_(0, 2 * π),
    p0=torch.empty(1, L, 1).normal_(),
    action=pullback,
    step_size=0.01,
    traj_length=4.,
)

# action_ = pullback(coords)
# forces = pullback.grad(coords).negative()
# data = torch.cat(
#     [
#         coords.squeeze(-1),
#         momentum.squeeze(-1),
#         action_,
#         forces.squeeze(-1),
#         forces.pow(2).sum(dim=1).sqrt(),
#     ],
#     dim=1,
# )

# trajectory = pd.DataFrame(
#     data=data,
#     index=time.numpy(),
#     columns=[f"x{i}" for i in range(L)] + [f"p{i}" for i in range(L)] + ["S"] + [f"F{i}" for i in range(L)] + ["|F|"],
# )

trajectory.head()

In [ ]:
trajectory.plot(y=["z1", "φ1"])

In [ ]:
trajectory.plot(y=["S_targ", "ldj", "S_pull", "|F|"], subplots=True, figsize=(10, 10))

In [ ]:
import seaborn as sns

sns.pairplot(trajectory[["S_targ", "|F|"]])

## Two dimensional experiment

In [ ]:
z = torch.linspace(0, 2 * π, 50)
z = torch.cartesian_prod(z, z).unsqueeze(-1)
with torch.no_grad():
    φ, ldj = model.flow(z)

S_pull = pullback(z)
F = pullback.grad(z).negative()
S_targ = pullback.target(φ)

data = torch.cat(
    [
        z.squeeze(-1),
        φ.squeeze(-1),
        F.squeeze(-1),
        F.pow(2).sum(1).sqrt(),
        ldj,
        S_pull,
        S_targ,
    ],
    dim=1,
)
columns = [
    *[f"z{i}" for i in range(L)],
    *[f"φ{i}" for i in range(L)],
    *[f"F{i}" for i in range(L)],
    "|F|",
    "ldj",
    "S_pull",
    "S_targ",
]

data = pd.DataFrame(
    data=data,
    columns=columns,
)
data.head()

In [ ]:
sns.heatmap(
    data.pivot(columns="z1", index="z0", values="S_targ")
)

In [ ]:
sns.heatmap(
    data.pivot(columns="z1", index="z0", values="S_pull")
)

In [ ]:
sns.scatterplot(
    data,
    x="φ0",
    y="φ1",
    s=3.
)

## Demonstration of the problem

In [ ]:
target = action(beta=6.0, lattice_size=2, lattice_dim=1)
flow = AutoregressiveFlow(
    lattice_size=2,
    n_mixture=1,
    net_shape=[16],
    net_activation="Tanh",
)
model = FlowBasedSampler(flow, target)

_ = model(1)  # instantiate lazy layers

model

In [ ]:
training_metrics = train(
    model, 
    n_steps=1000,
    batch_size=1024,
)

training_metrics.plot(x="step", y=["loss", "ess", "vlw"], subplots=True)

In [ ]:
pullback = PullbackAction(flow=model.flow, target=model.target)
time, coords, momentum = leapfrog(
    φ0=torch.empty(1, 8, 1).uniform_(0, 2 * π),
    p0=torch.empty(1, 8, 1).normal_(),
    action=pullback,
    step_size=0.01,
    traj_length=10.,
)

action_ = pullback(coords)
forces = pullback.grad(coords).negative()

trajectory = pd.DataFrame(
    data=torch.cat([action_, forces.pow(2).sum(dim=1)], dim=1).squeeze(-1),
    index=time.numpy(),
    columns=[S", "F"],
)

trajectory.head()

In [ ]:
trajectory.plot(subplots=True)

In [ ]:
import seaborn as sns

sns.pairplot(trajectory)

In [ ]:
trajectory.plot(y=["x1", "x2"], linestyle=":", marker="")

In [ ]:
trajectory.plot(y=["F1", "F2"], linestyle=":", marker="")